<a href="https://colab.research.google.com/github/GwonHyeokMIN/-/blob/main/%EA%B3%BC%EC%A0%9C2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
import zipfile
from keras import optimizers
from keras.models import Model

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/cats_and_dogs_filtered.zip'
data = zipfile.ZipFile(path,'r')
data.extractall('/tmp')
data.close()
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
train_set = train_datagen.flow_from_directory(
    '/tmp/cats_and_dogs_filtered/train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary')
test_set = test_datagen.flow_from_directory(
    '/tmp/cats_and_dogs_filtered/validation',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary')
train_step = train_set.n//32
test_step = test_set.n//32

In [ ]:
vgg16_base = tf.keras.applications.VGG16(include_top=False,
                                         input_shape=(224,224,3))
vgg16 = tf.keras.models.Sequential([
                                    vgg16_base,
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024,activation='relu'),
                                    tf.keras.layers.Dense(512,activation='relu'),
                                    tf.keras.layers.Dense(1,activation='sigmoid'),
])
vgg16.summary()

In [ ]:
vgg16.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

history = vgg16.fit_generator(train_set,
                              steps_per_epoch=train_step,
                              epochs=8,
                              validation_data=test_set,
                              validation_steps=test_step)

In [ ]:
fig,axs = plt.subplots(2,1,figsize=(15,15))
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'val'])
axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train','var'])